In [1]:
%pip install seqeval
from datasets import load_dataset
import os
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import json
from processing_function import *

You should consider upgrading via the '/Users/jlbt/boaworkspace/.env/dataset_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
ROOT = Path(os.path.abspath(""))

In [3]:
train = process_dataset(ROOT / 'dataset/train.txt')
validation = process_dataset(ROOT / 'dataset/valid.txt')
test = process_dataset(ROOT / 'dataset/test.txt')

In [4]:
#Save dataset in json format
for i in ['train','test','validation'] : 
    with open(f'{i}.json', "w") as json_file:
        json.dump(eval(i), json_file, indent=4)  # 'indent' for pretty formatting (optional)


In [5]:
#Load dataset
data_files = { "train": "train.json", "validation": "validation.json", "test": "test.json"}
dataset = load_dataset('json', data_files=data_files)

Using custom data configuration default-b9074ea3ec5dc791


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /Users/jlbt/.cache/huggingface/datasets/json/default-b9074ea3ec5dc791/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
from transformers import AutoTokenizer
ckpt = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(ckpt)

In [7]:
def encode_label(dataset):
    #Encode labels
    full_labels=[]
    for l in dataset['labels']:
        full_labels.extend(l)
    unique_labels = list(set(full_labels))
        
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(unique_labels)
    dataset['encoded_label'] = label_encoder.transform(dataset['labels'])
    return dataset

In [8]:
full_labels=[]
for l in dataset['train']:
    full_labels.extend(l['labels'])
unique_labels = list(set(full_labels))

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(unique_labels)
label_names = label_encoder.classes_


In [9]:
encode_datasets = dataset.map(
    encode_label,
    #batched=True,
    #remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/14987 [00:00<?, ?ex/s]

  0%|          | 0/3466 [00:00<?, ?ex/s]

  0%|          | 0/3684 [00:00<?, ?ex/s]

In [10]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label < 4 :
                label += 4
            new_labels.append(label)

    return new_labels

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['sentences'], truncation=True, is_split_into_words=True
    )
    all_labels = examples["encoded_label"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


In [12]:
tokenized_datasets = encode_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["encoded_label","sentences"],
)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14987
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3466
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3684
    })
})

In [14]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [15]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(4)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    4,    4,    4,    4,    4,    4, -100, -100, -100, -100],
        [-100,    1,    6,    1,    6,    6,    6,    1,    6,    6,    6, -100],
        [-100,    1,    3, -100, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100,    1,    5,    5,    5,    4,    4,    4,    4,    4, -100, -100]])

In [17]:
%pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 81 kB 8.0 MB/s eta 0:00:011
You should consider upgrading via the '/Users/jlbt/boaworkspace/.env/dataset_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import evaluate

metric = evaluate.load("seqeval")

In [19]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [20]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [21]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    ckpt,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [22]:
model.config.num_labels

9

In [25]:
%pip install --upgrade torch


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached torch-2.0.1-cp38-none-macosx_10_9_x86_64.whl (143.1 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.14.1 requires torch==1.13.1, but you'll

In [27]:
%pip install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 3.9 MB 6.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jlbt/boaworkspace/.env/dataset_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    use_mps_device =True,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/Users/jlbt/boaworkspace/.env/dataset_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14987
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 705
  Number of trainable parameters = 107726601


  0%|          | 0/705 [00:00<?, ?it/s]